## **Análisis de Datos de Ganado Bovino para Carne**
### **Introducción**
En este proyecto, realizaremos un análisis exploratorio de datos (EDA) y un tratamiento de valores faltantes en un dataset de ganado bovino destinado a la producción de carne. El objetivo es:

* ✅ Identificar valores faltantes y patrones en su distribución
* ✅ Analizar relaciones entre variables clave (edad, peso, raza, marmoleo)
* ✅ Aplicar técnicas de imputación adecuadas para cada caso
* ✅ Visualizar resultados para una mejor comprensión de los datos

Este análisis será útil para:

* **Productores ganaderos:** Mejorar el manejo de registros

* **Veterinarios:** Identificar patrones en el desarrollo del ganado

* **Analistas de datos:** Ejemplo práctico de limpieza de datos

### **Descripción del Dataset**
El dataframe contiene 7 columnas con información de ganado bovino para carne:

| Columna            | Tipo de dato | Descripción                     | Valores típicos               |
|--------------------|--------------|---------------------------------|-------------------------------|
| **ID_animal**      | `int`        | Identificador único del animal  | 101, 102, ..., 110            |
| **Edad_meses**     | `float`      | Edad del animal en meses        | 12, 24, 36, etc.              |
| **Peso_kg**        | `float`      | Peso vivo en kilogramos         | 320, 450, 600, etc.           |
| **Raza**           | `object`     | Raza del bovino                 | Angus, Hereford, Brahman       |
| **Grado_marmoleo** | `float`      | Calidad de carne (1-5)          | 2 (básico) a 5 (premium)      |
| **Vacunado**       | `bool`       | Estado de vacunación            | `True`/`False`                |
| **Tipo_alimentacion** | `object`  | Dieta del animal               | Pastoreo, Concentrado, Mixto  |


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Optional, List

# Dataframe para análisis
data = {
    'ID_animal': [101, 102, 103, 104, 105, 106, 107, 108, 109, 110],
    'Edad_meses': [24, 36, 18, np.nan, 30, 42, 12, 24, np.nan, 48],
    'Peso_kg': [450, 580, np.nan, 380, 520, 600, 320, np.nan, 410, 550],
    'Raza': ['Angus', 'Hereford', 'Brahman', 'Angus', np.nan, 'Hereford', 'Brahman', 'Angus', 'Hereford', 'Brahman'],
    'Grado_marmoleo': [3, 4, np.nan, 2, 5, 3, np.nan, 4, 3, 2],  # Escala de 1 a 5
    'Vacunado': [True, False, True, np.nan, True, False, np.nan, True, False, True],
    'Tipo_alimentacion': ['Pastoreo', 'Concentrado', 'Mixto', 'Pastoreo', 'Concentrado', np.nan, 'Mixto', 'Pastoreo', 'Concentrado', 'Mixto']
}

df_ganado = pd.DataFrame(data)
print(df_ganado.head())

   ID_animal  Edad_meses  Peso_kg      Raza  Grado_marmoleo Vacunado  \
0        101        24.0    450.0     Angus             3.0     True   
1        102        36.0    580.0  Hereford             4.0    False   
2        103        18.0      NaN   Brahman             NaN     True   
3        104         NaN    380.0     Angus             2.0      NaN   
4        105        30.0    520.0       NaN             5.0     True   

  Tipo_alimentacion  
0          Pastoreo  
1       Concentrado  
2             Mixto  
3          Pastoreo  
4       Concentrado  


Configuracion del estilo de gráfico personalizada

In [ ]:
%matplotlib inline
# Configuración de estilo
sns.set(style="whitegrid")
sns.set_context(context="notebook", font_scale=1.2)
plt.rcParams['figure.figsize'] = (6, 3)
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 12
# Configuración de paleta de colores
viridis_colors = sns.color_palette("viridis", as_cmap=False)
chosen_colors = viridis_colors[3]
chosen_colors2 = viridis_colors[5]
sns.set_palette(palette=viridis_colors)

Conocer cuantos valores no nulos tenemos en cada columna

In [ ]:
df_ganado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID_animal          10 non-null     int64  
 1   Edad_meses         8 non-null      float64
 2   Peso_kg            8 non-null      float64
 3   Raza               9 non-null      object 
 4   Grado_marmoleo     8 non-null      float64
 5   Vacunado           8 non-null      object 
 6   Tipo_alimentacion  9 non-null      object 
dtypes: float64(3), int64(1), object(3)
memory usage: 692.0+ bytes


Extendemos la API de pandas para poder personalizar pandas y utilizarlo en nuestro flujo de trabajo:
| Método                          | Descripción |
|---------------------------------|-------------|
| `number_missing()`              | Número total de valores faltantes |
| `number_complete()`             | Número total de valores completos |
| `percentage_missing()`          | Porcentaje de faltantes por columna |
| `missing_variable_summary()`    | Resumen de faltantes por variable (n_missing, pct_missing) |
| `missing_case_summary()`        | Resumen de casos con faltantes (n_missing, pct_missing) |
| `missing_variable_span()`       | Analiza faltantes en segmentos de datos (n_in_span, n_missing, pct_missing) |
| `sort_variables_by_missingness()` | Variables ordenadas por cantidad de faltantes |
| `create_shadow_matrix()`       | Matriz binaria de valores faltantes (1=missing, 0=present) |
| `bind_shadow_matrix()`         | Combina DataFrame original con shadow matrix |
| `missing_variable_plot()`      | Gráfico de barras: porcentaje de faltantes por variable |
| `missing_case_plot()`          | Heatmap de patrones de valores faltantes |
| `missing_patterns()`           | Frecuencia de patrones comunes de faltantes |

In [ ]:

@pd.api.extensions.register_dataframe_accessor("missing")
class MissingMethods:
    def __init__(self, pandas_obj):
        self._obj = pandas_obj
        self._set_plot_style()
    
    def _set_plot_style(self):
        """Configuración de estilo para gráficos"""
        sns.set(style="whitegrid", context="notebook", font_scale=1.2)
        plt.rcParams.update({
            'figure.figsize': (6, 3),
            'axes.titlesize': 16,
            'axes.labelsize': 14,
            'xtick.labelsize': 12,
            'ytick.labelsize': 12,
            'legend.fontsize': 12
        })
        self._colors = sns.color_palette("viridis")
    
    def number_missing(self) -> int:
        """Número total de valores faltantes."""
        return self._obj.isna().sum().sum()
    
    def number_complete(self) -> int:
        """Número total de valores completos."""
        return self._obj.size - self.number_missing()
    
    def percentage_missing(self) -> pd.Series:
        """Porcentaje de faltantes por columna."""
        return (self._obj.isna().mean() * 100).round(2)
    
    def missing_variable_summary(self) -> pd.DataFrame:
        """Resumen de faltantes por variable."""
        return (
            self._obj.isna()
            .sum()
            .reset_index(name="n_missing")
            .rename(columns={"index": "variable"})
            .assign(pct_missing=lambda df: df.n_missing / len(self._obj) * 100)
            .sort_values("pct_missing", ascending=False)
        )
    
    def missing_case_summary(self) -> pd.DataFrame:
        """Resumen de casos con faltantes."""
        return (
            self._obj.assign(
                n_missing=lambda df: df.isna().sum(axis=1),
                pct_missing=lambda df: df.n_missing / df.shape[1] * 100
            )
            .query("n_missing > 0")
            [["n_missing", "pct_missing"]]
            .reset_index()
            .rename(columns={"index": "case"})
        )
    
    def missing_variable_span(self, variable: str, span_every: int = 100) -> pd.DataFrame:
        """Analiza faltantes en segmentos de datos.
        
        Args:
            variable: Columna a analizar
            span_every: Tamaño del segmento (filas)
        """
        return (
            self._obj.assign(span=lambda df: np.arange(len(df)) // span_every)
            .groupby("span")
            [variable]
            .agg(
                n_in_span="size",
                n_missing=lambda x: x.isna().sum(),
                first="first",
                last="last"
            )
            .assign(
                pct_missing=lambda df: df.n_missing / df.n_in_span * 100,
                density=lambda df: df.n_in_span / span_every
            )
            .reset_index()
        )
    
    def sort_variables_by_missingness(self, ascending: bool = False) -> pd.Series:
        """Variables ordenadas por cantidad de faltantes."""
        return self._obj.isna().sum().sort_values(ascending=ascending)
    
    def create_shadow_matrix(self, suffix: str = "_shadow") -> pd.DataFrame:
        """Matriz binaria de valores faltantes."""
        return self._obj.isna().astype(int).add_suffix(suffix)
    
    def bind_shadow_matrix(self, suffix: str = "_shadow") -> pd.DataFrame:
        """Combina DataFrame original con shadow matrix."""
        return pd.concat([self._obj, self.create_shadow_matrix(suffix)], axis=1)
    
    def missing_variable_plot(self):
        """Gráfico de faltantes por variable."""
        df = self.missing_variable_summary()
        if df.empty:
            return
        
        plt.figure()
        sns.barplot(
            x="pct_missing", 
            y="variable", 
            data=df,
            color=self._colors[1]
        )
        plt.title("Valores faltantes por variable")
        plt.xlabel("Porcentaje faltante")
        plt.ylabel("Variable")
        plt.tight_layout()
        plt.show()
    
    def missing_case_plot(self, rotation: int = 45):
        """Heatmap de patrones de faltantes."""
        if not self._obj.isna().any().any():
            return
            
        plt.figure()
        sns.heatmap(
            self._obj.isna(),
            cbar=False,
            cmap=["white", self._colors[2]]
        )
        plt.title("Matriz de valores faltantes")
        plt.xlabel("Variables")
        plt.ylabel("Casos")
        plt.xticks(rotation=rotation)
        plt.tight_layout()
        plt.show()
    
    def missing_patterns(self) -> pd.DataFrame:
        """Identifica patrones comunes de faltantes."""
        return (
            self._obj.isna()
            .groupby(list(self._obj.columns))
            .size()
            .reset_index(name="count")
            .sort_values("count", ascending=False)
        )

/tmp/ipykernel_3299/517356932.py:5: UserWarning: registration of accessor <class '__main__.MissingMethods'> under name 'missing' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexisting attribute with the same name.
  @pd.api.extensions.register_dataframe_accessor("missing")


In [ ]:
df = pd.DataFrame(df_ganado)

In [ ]:
# Chequeamos los valores faltantes
df.isna()


,ID_animal,Edad_meses,Peso_kg,Raza,Grado_marmoleo,Vacunado,Tipo_alimentacion
0,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False
2,False,False,True,False,True,False,False
3,False,True,False,False,False,True,False
4,False,False,False,True,False,False,False
5,False,False,False,False,False,False,True
6,False,False,False,False,True,True,False
7,False,False,True,False,False,False,False
8,False,True,False,False,False,False,False
9,False,False,False,False,False,False,False


In [ ]:
# Resumen de los valores faltantes
print("Tenemos la siguiente cantidad de datos:",df.size,", con las siguientes filas y columnas",df.shape)

Tenemos la siguiente cantidad de datos: 70 , con las siguientes filas y columnas (10, 7)


In [ ]:
# Número total de valores completos o sea sin observaciones faltantes
print("Número total de valores completos:",df.missing.number_complete())
# Número total de valores faltantes
print("Número total de valores faltantes:",df.missing.number_missing())

Número total de valores completos: 60
Número total de valores faltantes: 10


In [ ]:
# Resumen tabular de los valores faltantes
df.missing.missing_variable_summary()

,Missing Values,Percentage Missing
Edad_meses,2,20.0
Peso_kg,2,20.0
Grado_marmoleo,2,20.0
Vacunado,2,20.0
Raza,1,10.0
Tipo_alimentacion,1,10.0


In [ ]:
# Observaciones con valores faltantes
df.missing.missing_case_summary()

,ID_animal,Edad_meses,Peso_kg,Raza,Grado_marmoleo,Vacunado,Tipo_alimentacion
0,103,18.0,NaN,Brahman,NaN,True,Mixto
1,104,NaN,380.0,Angus,2.0,NaN,Pastoreo
2,105,30.0,520.0,NaN,5.0,True,Concentrado
3,106,42.0,600.0,Hereford,3.0,False,NaN
4,107,12.0,320.0,Brahman,NaN,NaN,Mixto
5,108,24.0,NaN,Angus,4.0,True,Pastoreo
6,109,NaN,410.0,Hereford,3.0,False,Concentrado


In [ ]:
(
    df
    .missing
    .missing_variable_span(
        variable="Peso_kg",
        span_every=50
    )
)

,segment,total_values,missing_values,percentage_missing,values_density,first_value,last_value
0,0,8,2,25.0,0.16,450.0,550.0
